<a href="https://colab.research.google.com/github/Hikachhu/ProgGPU/blob/main/CorrigeTD1ProgGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Executez cette cellule pour installer la bonne version de CUDA

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

# Exercise 3

In [ ]:
%%cu
#include <iostream>
__global__ void kernel(void){
    printf("Hello world from block %d, thread %d\n", blockIdx.x, threadIdx.x);
}

int main(void){
    kernel<<<10,10>>>();
    cudaDeviceSynchronize();
    return 0;
}

# Exercise 4

In [6]:
%%cu
#include <iostream>
#include <chrono>

#define N 512
#define Niter 1000

void addCPU(int *a, int *b, int *c) {
    for (int i = 0; i < N*N ; i++) {
        c[i] = a[i] + b[i];
    }
}

__global__ void addKernel1D (int *a, int *b, int *c) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    while (tid < N*N) {
        c[tid] = a[tid] + b[tid];
        tid += blockDim.x * gridDim.x;
    }
}

int main(void) {
  int A[N*N], B[N*N], C[N*N], D[N*N];
  int *d_A, *d_B, *d_C;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaMalloc(&d_A, N*N*sizeof(int));
  cudaMalloc(&d_B, N*N*sizeof(int));
  cudaMalloc(&d_C, N*N*sizeof(int));

  for (int i = 0 ; i < N*N ; i++) {
          A[i] = i;
          B[i] = i+1;
  }
  cudaMemcpy(d_A, A, N*N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, N*N*sizeof(int), cudaMemcpyHostToDevice);

  auto t_start = std::chrono::high_resolution_clock::now();
  for (int i = 0; i <  Niter ; i++) // We do Niter to have an averaged execution time
    addCPU(A, B, C);
  auto t_end = std::chrono::high_resolution_clock::now();

  cudaDeviceProp properties;
  cudaGetDeviceProperties(&properties, 0);
  int nThreads = properties.maxThreadsPerBlock; // Get maximum number of threads per block
  int nBlocks = (N*N + nThreads - 1) / nThreads; // Get right number of blocks to cover our problem
  addKernel1D<<<nBlocks,nThreads>>>(d_A, d_B, d_C); // WARM-UP
  cudaEventRecord(start);
  for (int i = 0; i <  Niter ; i++) // We do Niter to have an averaged execution time
    addKernel1D<<<nBlocks,nThreads>>>(d_A, d_B, d_C);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  float ms;
  cudaEventElapsedTime(&ms, start, stop);

  cudaMemcpy(D, d_C, N*N*sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(B, d_B, N*N*sizeof(int), cudaMemcpyDeviceToHost);

  int diff = 0;
  for (int i = 0 ; i < N ; i++) {
      if(D[i] != C[i]) diff = D[i] - C[i];
  }
  if(diff != 0) {
      printf("Wrong computation : diff = %d", diff);
      return 0;
  }
  printf("CPU execution time = %f ms\n", std::chrono::duration<double, std::milli>(t_end-t_start).count()/Niter);
  printf("GPU execution time = %f ms\n", ms/Niter);
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

}

CPU execution time = 0.758210 ms
GPU execution time = 0.011899 ms

